<a href="https://colab.research.google.com/github/janbanot/msc-cs-code/blob/main/sem3/DL/DL_2025_Lab5-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%html
<style>
.cell-output-ipywidget-background{background-color:transparent!important}
:root{--jp-widgets-color:var(--vscode-editor-foreground);--jp-widgets-font-size:var(--vscode-editor-font-size)}
</style>

# Przykład 1

Wytrenowany model językowy typu BERT można zastosować, np. do przewidywania **brakujących słów w tekście** -- tutaj oznaczanych za pomocą specjalnego tokena [MASK].

In [ ]:
import torch
from transformers import BertTokenizer, BertForMaskedLM

# Załaduj wstępnie wytrenowany model i tokenizator
# To inicjalizuje tokenizator BERT i model języka maskowanego z wersji wstępnie wytrenowanej
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.eval()  # Ustawia model w tryb ewaluacji, wyłączając dropout i normalizację wsadową

print(f"L. param. modelu: {sum(p.numel() for p in model.parameters()) / 1_000_000:.1f} mln")

In [ ]:
# Przykładowe zdanie z zamaskowanym tokenem
sentence = "The quick brown [MASK] jumps over the lazy dog."

# Tokenizuj wejście
# Przekształca zdanie w identyfikatory tokenów, dodając specjalne tokeny i zwracając tensor PyTorch
input_ids = tokenizer.encode(sentence, return_tensors='pt')
print(f'{input_ids = }')

# Znajduje indeks tokenu [MASK] w stokenizowanym wejściu
mask_token_index = torch.where(input_ids == tokenizer.mask_token_id)[1]
print(f'{tokenizer.mask_token_id = } {mask_token_index = }')

# Uzyskaj przewidywania dla zamaskowanego tokenu
with torch.no_grad():  # wyłącz obliczanie gradientów, aby zaoszczędzić pamięć podczas wnioskowania
    output = model(input_ids)
logits = output.logits  # Surowe wyniki predykcji dla każdego tokenu w słowniku

# Wyodrębnij logity odpowiadające zamaskowanemu tokenowi i uzyskaj 5 najlepszych przewidywań
mask_token_logits = logits[0, mask_token_index, :]
# Pobiera indeksy 5 tokenów o najwyższych wynikach
top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()
print(f'{top_5_tokens = }')

# Dekoduj przewidywane tokeny
# Przekształca identyfikatory tokenów z powrotem w czytelne dla człowieka słowa, usuwając białe znaki
predicted_tokens = [tokenizer.decode([token]).strip() for token in top_5_tokens]
print("Przewidywane tokeny:", predicted_tokens)

In [ ]:
# Możemy również zamienić logity na prawdop. za pomocą softmax
probs = torch.softmax(mask_token_logits, dim=1)
top_k = torch.topk(probs, k=5)
top_confidence, top_idx = top_k.values, top_k.indices

for token, prob in zip(top_idx[0], top_confidence[0]):
    print(f"{tokenizer.decode([token]).strip()} - {prob.item():.4f}")

# Zadanie 1

Na podstawie poprzedniego przykładu sprawdź jak model BERT radzi sobie z przewidywaniem różnych części mowy w podanych zdaniach.

Użyj modelu `dkleczek/bert-base-polish-uncased-v1` dla języka polskiego.

Zdania:

    sentences = {
        "noun": "Uwielbiam jeść [MASK].",
        "verb": "Ja [MASK] pizzę.",
        "adjective": "Ta pizza jest [MASK]."
    }

## Rozwiązanie

In [ ]:
tokenizer = BertTokenizer.from_pretrained('dkleczek/bert-base-polish-uncased-v1')
model = BertForMaskedLM.from_pretrained('dkleczek/bert-base-polish-uncased-v1')


print(f"L. param. modelu: {sum(p.numel() for p in model.parameters()) / 1_000_000:.1f} mln")

In [ ]:
sentences = {
    "noun": "Uwielbiam jeść małe [MASK].",
    "verb": "Ja [MASK] małą pizzę.",
    "adjective": "Ten chłopak jest [MASK]."
}

for part_of_speech, sentence in sentences.items():
    print(f"\nPrzewidywania dla '{part_of_speech}':")
    # Tokenizuj wejście
    input_ids = tokenizer.encode(sentence, return_tensors='pt')

    # Znajduje indeks tokenu [MASK] w stokenizowanym wejściu
    mask_token_index = torch.where(input_ids == tokenizer.mask_token_id)[1]

    # Uzyskaj przewidywania dla zamaskowanego tokenu
    with torch.no_grad():
        output = model(input_ids)
    logits = output.logits

    # Wyodrębnij logity odpowiadające zamaskowanemu tokenowi i uzyskaj 5 najlepszych przewidywań
    mask_token_logits = logits[0, mask_token_index, :]
    top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

    # Dekoduj przewidywane tokeny
    predicted_tokens = [tokenizer.decode([token]).strip() for token in top_5_tokens]
    print("Przewidywane tokeny:", predicted_tokens)
